# Notebook for final pfoject of IBM's professional Data Science specialization

In [226]:
import pandas as pd
import numpy as np
from pandas.io.html import read_html

In [262]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
DFs = read_html(page, match='Borough', header=0) # a list of dataframes
df = DFs[0]#getting the dataframe frm the list

## Exploring the data:

In [263]:
df.describe()

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M9V,Not assigned,Not assigned
freq,8,77,78


In [264]:
print('not assigned boroughs: ', df.loc[df.Borough == 'Not assigned', 'Borough'].count())
print('not assigned neighbourhoods: ', df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'].count())

not assigned boroughs:  77
not assigned neighbourhoods:  78


### Ignore cells with a borough that is Not assigned:

In [265]:
df.Borough.replace('Not assigned', np.nan, inplace=True)

In [266]:
df.dropna(subset=['Borough'], inplace=True)
print('not assigned boroughs: ', df.loc[df.Borough == 'Not assigned', 'Borough'].count())

not assigned boroughs:  0


### grouping repeated neighbourhoods/postcode:

In [267]:
df.set_index(['Postcode', 'Borough'], inplace=True)

In [268]:
d = df.groupby(level=['Postcode','Borough']).agg(','.join)
d.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


### processing not assigned neighbourhoods:

In [269]:
d.reset_index(level=['Borough'], inplace=True)
d.columns

Index(['Borough', 'Neighbourhood'], dtype='object')

In [270]:
d.loc['M1B']

Borough            Scarborough
Neighbourhood    Rouge,Malvern
Name: M1B, dtype: object

In [275]:
print('not assigned neighbourhoods: ', d.loc[d.Neighbourhood == 'Not assigned', 'Neighbourhood'].count())


not assigned neighbourhoods:  0


In [276]:
d.loc[d.Neighbourhood == 'Not assigned', 'Borough']

Series([], Name: Borough, dtype: object)

In [273]:
d.loc[d.Neighbourhood == 'Not assigned', 'Neighbourhood'] = 'Queen\'s park'

In [274]:
d.shape

(103, 2)

### Getting longitude and latitude of postcodes:

In [240]:
import geocoder

In [75]:
latitude = []
longitude = []
coordinates = None
for code in df.Postcode:
    while (coordinates == None):
        coordinates = geocoder.google(code + ', Toronto, Ontario').latlng
    latitude.append(coordinates[0])
    longitude.append(coordinates[1])

KeyboardInterrupt: 

##### the geocoder package is very un reliable reliable, hence I will use a file that provides the required coordinates

In [277]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [278]:
d['Latitude'] = 0
d['Longitude'] = 0

In [279]:
for code in d.index:
    if code in coordinates['Postal Code'].values:
        d.loc[code, 'Latitude'] = coordinates.loc[coordinates['Postal Code'] == code, 'Latitude'].values
        d.loc[code, 'Longitude'] = coordinates.loc[coordinates['Postal Code'] == code, 'Longitude'].values        

In [280]:
d.head(40)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


In [281]:
d.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


## Putting data on map:


In [282]:
import folium

In [283]:
#Toronto cental location is [43.70, -79.42 ]

# create map of Toronot using latitude and longitude values
toronot_map = folium.Map(location=[43.70, -79.42 ], zoom_start=11.5)
 
#add markers of neighbourhoods
for lat, lng, label, code in zip(d['Latitude'], d['Longitude'], d['Neighbourhood'], d.index):
    label = folium.Popup(label, parse_html=True)
    code = folium.Popup(code, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='red',
        fill=True).add_to(toronot_map) 
    
    
toronot_map

### Define Foursquare Credentials and Version:

In [287]:
CLIENT_ID = ***
CLIENT_SECRET = ***
VERSION = '20180605' # Foursquare API version


In [285]:
# function that extracts the category of the venue that would be requested from Foursquare
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [286]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [292]:
venues = getNearbyVenues(names=d['Neighbourhood'],
                                   latitudes=d['Latitude'],
                                   longitudes=d['Longitude']
                                  )

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=MWFD5ILZQJ3254JQYSXS2HDV4A0ENLR01KH5G0E0CO4UIF5R&client_secret=E4J44JBKVOTPVE5LVCM4ARTYAINR3IBQLA4WV2GXDWPU0ZPZ&v=20180605&ll=43.716316,-79.23947609999999&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f1decfd5780>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [291]:
venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
5,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
6,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
7,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
8,"Guildwood,Morningside,West Hill",43.763573,-79.188711,chatr Mobile,43.765917,-79.191672,Tech Startup
9,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
